In [28]:
import numpy as np
import pandas as pd
from pathlib import Path
from skimage import io, color, filters, exposure

In [29]:
BASE_DIR = Path("Retina_Vessel/training")
IMAGE_DIR = BASE_DIR / "images"
GT_DIR = BASE_DIR / "1st_manual"
ROI_DIR = BASE_DIR / "mask"

In [30]:
def load_image(image_path):
    """
    Load image, extract green channel, normalize, and invert.
    
    Why green channel?
    Retinal vessels have maximum contrast in green channel.
    """
    img = io.imread(image_path)

    # Extract green channel (DRIVE images are RGB)
    green = img[:, :, 1]

    # Normalize to [0,1]
    green = exposure.rescale_intensity(green.astype(float))

    # Invert because vessels are darker than background
    green = 1.0 - green

    return green

Load mask and convert to binary (0,1).

In [31]:
def load_binary_mask(path):
    
    mask = io.imread(path)
    mask = mask.squeeze()  # remove singleton dimensions

    if mask.ndim == 3:
        mask = color.rgb2gray(mask)  # assume RGB if 3D

    return (mask > 127).astype(np.uint8)

### Applying local adaptive thresholding.

    Niblack:
        T = m + k*s

    Sauvola:
        T = m * (1 + k * (s/R - 1))

    where:
        m = local mean
        s = local std
        R = dynamic range (default 128)

In [32]:
def adaptive_threshold(image, method="sauvola", window_size=35, k=0.2):
    
    if method == "niblack":
        thresh = filters.threshold_niblack(image, window_size, k)
    elif method == "sauvola":
        thresh = filters.threshold_sauvola(image, window_size, k)
    else:
        raise ValueError("Method must be 'niblack' or 'sauvola'")

    return (image > thresh).astype(np.uint8)

### Compute:
    - Sensitivity

In [33]:
def compute_metrics(pred, gt, roi):

    # Evaluate only inside ROI
    pred = pred[roi == 1]
    gt = gt[roi == 1]

    tp = np.sum((pred == 1) & (gt == 1))
    tn = np.sum((pred == 0) & (gt == 0))
    fp = np.sum((pred == 1) & (gt == 0))
    fn = np.sum((pred == 0) & (gt == 1))

    sensitivity = tp / (tp + fn)

    return sensitivity

In [34]:
results = []

image_files = sorted(IMAGE_DIR.glob("*.tif"))

for img_path in image_files:
    stem = img_path.stem.split("_")[0]

    gt_path = GT_DIR / f"{stem}_manual1.gif"
    roi_path = ROI_DIR / f"{stem}_training_mask.gif"

    # Load data
    image = load_image(img_path)
    gt = load_binary_mask(gt_path)
    roi = load_binary_mask(roi_path)

    for method in ["niblack", "sauvola"]:
        pred = adaptive_threshold(image, method=method)
        pred = pred * roi

        sens = compute_metrics(pred, gt, roi)

        results.append({
            "Image": stem,
            "Method": method,
            "Sensitivity": sens
        })
        
df = pd.DataFrame(results)

In [36]:
summary = df.groupby("Method").mean(numeric_only=True)

print("\nComparing Sensitivity of niblack and sauvola:\n")
print(summary)


Comparing Sensitivity of niblack and sauvola:

         Sensitivity
Method              
niblack     0.891522
sauvola     0.984418
